In [5]:
!pip install snowflake-connector-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 200.1 kB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 2.0 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 kB 404.5 kB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 425.2 kB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 2.0 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.7/85.7 kB 293.8 kB/s eta 0:00:000:00:01


In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("IngestNYCTaxiData") \
    .config("spark.jars", "/home/jovyan/jars/snowflake-jdbc-3.13.30.jar,/home/jovyan/jars/spark-snowflake_2.12-2.12.0-spark_3.4.jar") \
    .getOrCreate()
#en la sesión de Spark se incluye los archivos jar

In [ ]:
#obtener credenciales y guardarlas
import os

sfOptions = {
    "sfURL": f"https://{os.getenv('SF_ACCOUNT')}.snowflakecomputing.com",
    "sfUser": os.getenv("SF_USER"),
    "sfPassword": os.getenv("SF_PASSWORD"),
    "sfWarehouse": os.getenv("SF_WAREHOUSE"),
    "sfDatabase": os.getenv("SF_DATABASE"),
    "sfSchema": os.getenv("SF_SCHEMA"),
    "sfRole": os.getenv("SF_ROLE")
}

In [ ]:
# ============================================
# A) TAXI_ZONES (download CSV locally → load to SILVER)
# ============================================
zones_url  = "https://d37ci6vzurychx.cloudfront.net/misc/taxi_zone_lookup.csv"
zones_path = "/home/work/taxi_zone_lookup.csv"

# Download if not present
if not os.path.exists(zones_path):
    urllib.request.urlretrieve(zones_url, zones_path)

# Read CSV
zones_df = (
    spark.read
    .option("header", True)
    .option("inferSchema", True)
    .csv(zones_path)
)

# Select & cast with required names, replace NULLs with "Unknown"
zones_df = (
    zones_df
    .select(
        F.col("LocationID").cast(IntegerType()).alias("LocationID"),
        F.col("Borough").cast(StringType()).alias("Borough"),
        F.col("Zone").cast(StringType()).alias("Zone"),
        F.col("service_zone").cast(StringType()).alias("service_zone"),
    )
    .fillna({"Borough": "Unknown", "Zone": "Unknown", "service_zone": "Unknown"})
)

# Write to SILVER.TAXI_ZONES 
(
    zones_df.write
    .format("snowflake")
    .options(**sfOptions)
    .option("dbtable", "SILVER.TAXI_ZONES")
    .mode("overwrite")
    .save()
)


In [14]:
import os
import calendar
import snowflake.connector

START_YEAR, START_MONTH = 2015, 1
END_YEAR,   END_MONTH   = 2025, 8

conn = snowflake.connector.connect(
    account=os.getenv("SF_ACCOUNT"),
    user=os.getenv("SF_USER"),
    password=os.getenv("SF_PASSWORD"),
    warehouse=os.getenv("SF_WAREHOUSE"),
    database=os.getenv("SF_DATABASE"),
    schema="PUBLIC",
    role=os.getenv("SF_ROLE"),
)
cur = conn.cursor()

def run(sql):
    cur.execute(sql)
    return cur

# 0) Esquema y tabla base (si no existen)
run("""CREATE SCHEMA IF NOT EXISTS SILVER;""")
run("""
CREATE TABLE IF NOT EXISTS SILVER.NYC_ALL_TAXITRIPS (
    HASH_KEY                STRING,
    PICKUP_DATETIME         TIMESTAMP_NTZ,
    DROPOFF_DATETIME        TIMESTAMP_NTZ,
    VENDORID                NUMBER(38,0),
    VENDOR_DESC             STRING,
    RATECODEID              FLOAT,
    RATE_CODE_DESC          STRING,
    PASSENGER_COUNT         FLOAT,
    TRIP_DISTANCE           FLOAT,
    PULOCATIONID            NUMBER(38,0),
    DOLOCATIONID            NUMBER(38,0),
    STORE_AND_FWD_FLAG      STRING,
    PAYMENT_TYPE            FLOAT,
    PAYMENT_TYPE_DESC       STRING,
    FARE_AMOUNT             FLOAT,
    EXTRA                   FLOAT,
    MTA_TAX                 FLOAT,
    TIP_AMOUNT              FLOAT,
    TOLLS_AMOUNT            FLOAT,
    IMPROVEMENT_SURCHARGE   FLOAT,
    TOTAL_AMOUNT            FLOAT,
    CONGESTION_SURCHARGE    FLOAT,
    CBD_CONGESTION_FEE      FLOAT,
    AIRPORT_FEE             FLOAT,
    EHAIL_FEE               FLOAT,
    TRIP_TYPE               FLOAT,
    SERVICE_TYPE            STRING,
    SOURCE_YEAR             NUMBER(38,0),
    SOURCE_MONTH            NUMBER(38,0),
    RUN_ID                  STRING,
    INGESTED_AT_UTC         TIMESTAMP_NTZ,
    SOURCE_PATH             STRING
);
""")

# Templates de inserción (anti-dup con NOT EXISTS por HASH_KEY)
YELLOW_INSERT_TMPL = """
INSERT INTO SILVER.NYC_ALL_TAXITRIPS
SELECT *
FROM (
    SELECT
        MD5(
            COALESCE(TO_CHAR(tpep_pickup_datetime, 'YYYY-MM-DD HH24:MI:SS.FF3'),'') || '|' ||
            COALESCE(TO_CHAR(tpep_dropoff_datetime,'YYYY-MM-DD HH24:MI:SS.FF3'),'') || '|' ||
            COALESCE(TO_VARCHAR(VendorID),'') || '|' ||
            COALESCE(TO_VARCHAR(PULocationID),'') || '|' ||
            COALESCE(TO_VARCHAR(DOLocationID),'') || '|' ||
            COALESCE(TO_VARCHAR(total_amount),'') || '|' ||
            'YELLOW'
        ) AS HASH_KEY,
        tpep_pickup_datetime::TIMESTAMP_NTZ  AS PICKUP_DATETIME,
        tpep_dropoff_datetime::TIMESTAMP_NTZ AS DROPOFF_DATETIME,
        VendorID                              AS VENDORID,
        CASE VendorID
            WHEN 1 THEN 'Creative Mobile Technologies (CMT)'
            WHEN 2 THEN 'VeriFone Transportation Systems (VTS)'
            WHEN 4 THEN 'DDS'
            WHEN 5 THEN 'VIP'
            ELSE 'Unknown'
        END AS VENDOR_DESC,
        RatecodeID                            AS RATECODEID,
        CASE RatecodeID
            WHEN 1 THEN 'Standard rate'
            WHEN 2 THEN 'JFK'
            WHEN 3 THEN 'Newark'
            WHEN 4 THEN 'Nassau/Westchester'
            WHEN 5 THEN 'Negotiated fare'
            WHEN 6 THEN 'Group ride'
            ELSE 'Unknown'
        END AS RATE_CODE_DESC,
        passenger_count                       AS PASSENGER_COUNT,
        trip_distance                         AS TRIP_DISTANCE,
        PULocationID                          AS PULOCATIONID,
        DOLocationID                          AS DOLOCATIONID,
        store_and_fwd_flag                    AS STORE_AND_FWD_FLAG,
        payment_type                          AS PAYMENT_TYPE,
        CASE payment_type
            WHEN 1 THEN 'Credit card'
            WHEN 2 THEN 'Cash'
            WHEN 3 THEN 'No charge'
            WHEN 4 THEN 'Dispute'
            WHEN 5 THEN 'Unknown'
            WHEN 6 THEN 'Voided trip'
            ELSE 'Unknown'
        END AS PAYMENT_TYPE_DESC,
        fare_amount                           AS FARE_AMOUNT,
        extra                                 AS EXTRA,
        mta_tax                               AS MTA_TAX,
        tip_amount                            AS TIP_AMOUNT,
        tolls_amount                          AS TOLLS_AMOUNT,
        improvement_surcharge                 AS IMPROVEMENT_SURCHARGE,
        total_amount                          AS TOTAL_AMOUNT,
        congestion_surcharge                  AS CONGESTION_SURCHARGE,
        cbd_congestion_fee                    AS CBD_CONGESTION_FEE,
        airport_fee                           AS AIRPORT_FEE,          -- solo Yellow
        CAST(NULL AS FLOAT)                   AS EHAIL_FEE,
        CAST(NULL AS FLOAT)                   AS TRIP_TYPE,
        'YELLOW'                              AS SERVICE_TYPE,
        SOURCE_YEAR, SOURCE_MONTH, RUN_ID, INGESTED_AT_UTC, SOURCE_PATH
    FROM RAW.NYC_YELLOW_TAXIS
    WHERE SOURCE_YEAR = {yy} AND SOURCE_MONTH = {mm}
) S
WHERE NOT EXISTS (
    SELECT 1 FROM SILVER.NYC_ALL_TAXITRIPS T WHERE T.HASH_KEY = S.HASH_KEY
);
"""

GREEN_INSERT_TMPL = """
INSERT INTO SILVER.NYC_ALL_TAXITRIPS
SELECT *
FROM (
    SELECT
        MD5(
            COALESCE(TO_CHAR(lpep_pickup_datetime, 'YYYY-MM-DD HH24:MI:SS.FF3'),'') || '|' ||
            COALESCE(TO_CHAR(lpep_dropoff_datetime,'YYYY-MM-DD HH24:MI:SS.FF3'),'') || '|' ||
            COALESCE(TO_VARCHAR(VendorID),'') || '|' ||
            COALESCE(TO_VARCHAR(PULocationID),'') || '|' ||
            COALESCE(TO_VARCHAR(DOLocationID),'') || '|' ||
            COALESCE(TO_VARCHAR(total_amount),'') || '|' ||
            'GREEN'
        ) AS HASH_KEY,
        lpep_pickup_datetime::TIMESTAMP_NTZ   AS PICKUP_DATETIME,
        lpep_dropoff_datetime::TIMESTAMP_NTZ  AS DROPOFF_DATETIME,
        VendorID                              AS VENDORID,
        CASE VendorID
            WHEN 1 THEN 'Creative Mobile Technologies (CMT)'
            WHEN 2 THEN 'VeriFone Transportation Systems (VTS)'
            WHEN 4 THEN 'DDS'
            WHEN 5 THEN 'VIP'
            ELSE 'Unknown'
        END AS VENDOR_DESC,
        RatecodeID                            AS RATECODEID,
        CASE RatecodeID
            WHEN 1 THEN 'Standard rate'
            WHEN 2 THEN 'JFK'
            WHEN 3 THEN 'Newark'
            WHEN 4 THEN 'Nassau/Westchester'
            WHEN 5 THEN 'Negotiated fare'
            WHEN 6 THEN 'Group ride'
            ELSE 'Unknown'
        END AS RATE_CODE_DESC,
        passenger_count                       AS PASSENGER_COUNT,
        trip_distance                         AS TRIP_DISTANCE,
        PULocationID                          AS PULOCATIONID,
        DOLocationID                          AS DOLOCATIONID,
        store_and_fwd_flag                    AS STORE_AND_FWD_FLAG,
        payment_type                          AS PAYMENT_TYPE,
        CASE payment_type
            WHEN 1 THEN 'Credit card'
            WHEN 2 THEN 'Cash'
            WHEN 3 THEN 'No charge'
            WHEN 4 THEN 'Dispute'
            WHEN 5 THEN 'Unknown'
            WHEN 6 THEN 'Voided trip'
            ELSE 'Unknown'
        END AS PAYMENT_TYPE_DESC,
        fare_amount                           AS FARE_AMOUNT,
        extra                                 AS EXTRA,
        mta_tax                               AS MTA_TAX,
        tip_amount                            AS TIP_AMOUNT,
        tolls_amount                          AS TOLLS_AMOUNT,
        improvement_surcharge                 AS IMPROVEMENT_SURCHARGE,
        total_amount                          AS TOTAL_AMOUNT,
        congestion_surcharge                  AS CONGESTION_SURCHARGE,
        cbd_congestion_fee                    AS CBD_CONGESTION_FEE,
        CAST(NULL AS FLOAT)                   AS AIRPORT_FEE,          -- no existe en Green
        ehail_fee                             AS EHAIL_FEE,
        trip_type                             AS TRIP_TYPE,
        'GREEN'                               AS SERVICE_TYPE,
        SOURCE_YEAR, SOURCE_MONTH, RUN_ID, INGESTED_AT_UTC, SOURCE_PATH
    FROM RAW.NYC_GREEN_TAXIS
    WHERE SOURCE_YEAR = {yy} AND SOURCE_MONTH = {mm}
) S
WHERE NOT EXISTS (
    SELECT 1 FROM SILVER.NYC_ALL_TAXITRIPS T WHERE T.HASH_KEY = S.HASH_KEY
);
"""

def iter_months(y0, m0, y1, m1):
    y, m = y0, m0
    yield y, m
    while (y, m) != (y1, m1):
        if m == 12:
            y, m = y + 1, 1
        else:
            m += 1
        yield y, m

# 1) Batch insert mes a mes (VERBOSE)
for yy, mm in iter_months(START_YEAR, START_MONTH, END_YEAR, END_MONTH):
    tag = f"{yy}-{mm:02d}"
    try:
        print(f"🔥 Insertando YELLOW {tag} ...")
        cur.execute(YELLOW_INSERT_TMPL.format(yy=yy, mm=mm))
        print(f"   ✅ YELLOW {tag} - filas insertadas: {cur.rowcount}")
    except Exception as e:
        print(f"   ❌ YELLOW {tag} falló: {e}")

    try:
        print(f"🔥 Insertando GREEN  {tag} ...")
        cur.execute(GREEN_INSERT_TMPL.format(yy=yy, mm=mm))
        print(f"   ✅ GREEN  {tag} - filas insertadas: {cur.rowcount}")
    except Exception as e:
        print(f"   ❌ GREEN  {tag} falló: {e}")

# 2) Construir tabla enriquecida (JOIN con Taxi Zones) — tabla aparte
print("🧩 Construyendo SILVER.NYC_ALL_TAXIS_ENRICHED ...")
run("""
CREATE OR REPLACE TABLE SILVER.NYC_ALL_TAXIS_ENRICHED AS
SELECT
    t.*,
    pu.Borough AS PICKUP_BOROUGH,
    pu.Zone    AS PICKUP_ZONE,
    do.Borough AS DROPOFF_BOROUGH,
    do.Zone    AS DROPOFF_ZONE
FROM SILVER.NYC_ALL_TAXITRIPS t
LEFT JOIN SILVER.TAXI_ZONES pu ON t.PULocationID = pu.LocationID
LEFT JOIN SILVER.TAXI_ZONES do ON t.DOLocationID = do.LocationID;
""")
print("✅ SILVER.NYC_ALL_TAXIS_ENRICHED creada.")

# 3) Totales
cur.execute("SELECT COUNT(*) FROM SILVER.NYC_ALL_TAXITRIPS;")
print("📦 Total filas en SILVER.NYC_ALL_TAXITRIPS:", cur.fetchone()[0])
cur.execute("SELECT COUNT(*) FROM SILVER.NYC_ALL_TAXIS_ENRICHED;")
print("📦 Total filas en SILVER.NYC_ALL_TAXIS_ENRICHED:", cur.fetchone()[0])

cur.close()
conn.close()
print("🏁 Batch SILVER finalizado (verbose).")


🔥 Insertando YELLOW 2015-01 ...
   ✅ YELLOW 2015-01 - filas insertadas: 0
🔥 Insertando GREEN  2015-01 ...
   ✅ GREEN  2015-01 - filas insertadas: 0
🔥 Insertando YELLOW 2015-02 ...
   ✅ YELLOW 2015-02 - filas insertadas: 0
🔥 Insertando GREEN  2015-02 ...
   ✅ GREEN  2015-02 - filas insertadas: 0
🔥 Insertando YELLOW 2015-03 ...
   ✅ YELLOW 2015-03 - filas insertadas: 13342951
🔥 Insertando GREEN  2015-03 ...
   ✅ GREEN  2015-03 - filas insertadas: 0
🔥 Insertando YELLOW 2015-04 ...
   ✅ YELLOW 2015-04 - filas insertadas: 13063758
🔥 Insertando GREEN  2015-04 ...
   ✅ GREEN  2015-04 - filas insertadas: 0
🔥 Insertando YELLOW 2015-05 ...
   ✅ YELLOW 2015-05 - filas insertadas: 13157677
🔥 Insertando GREEN  2015-05 ...
   ✅ GREEN  2015-05 - filas insertadas: 1786848
🔥 Insertando YELLOW 2015-06 ...
   ✅ YELLOW 2015-06 - filas insertadas: 12324936
🔥 Insertando GREEN  2015-06 ...
   ✅ GREEN  2015-06 - filas insertadas: 1638868
🔥 Insertando YELLOW 2015-07 ...
   ✅ YELLOW 2015-07 - filas insertadas: 